In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

# Carga y Limpieza de Datos
yucatan_df = pd.read_csv("iter_31_cpv2010.csv", encoding="latin-1", low_memory=False)

# Columnas numéricas para limpieza específica
cols_num = ["pobtot", "pobmas", "pobfem", "pocupada_m", "pocupada_f", "pdesocup_m", "pdesocup_f", "hogjef_m", "hogjef_f"]

# Reemplazar " " y "*" solo en columnas numéricas
yucatan_df[cols_num] = yucatan_df[cols_num].replace([" ", "*"], np.nan)

# Convertir columnas numéricas a float (para manejar NaN)
yucatan_df[cols_num] = yucatan_df[cols_num].apply(pd.to_numeric, errors='coerce')

# Eliminar filas con NaN en columnas numéricas clave (ajustar subset según necesidades)
yucatan_df.dropna(subset=cols_num, inplace=True)
yucatan_df = yucatan_df.reset_index(drop=True)
yucatan_df = yucatan_df[["mun", "loc", "nom_ent", "nom_mun", "pobtot", "pobmas", "pobfem", "pocupada_m", "pocupada_f", "pdesocup_m", "pdesocup_f", "hogjef_m", "hogjef_f"]]

def info():
    print(yucatan_df.head(10))
    print(yucatan_df.info())
    print(yucatan_df.isnull().sum())
    print(yucatan_df.describe())  # Agregado para resúmenes generales

# Análisis de datos en Yucatán 2010 (General)
def analisis_general(yucatan_df):
    print("Análisis General de Yucatán 2010")
    
    try:
        n = int(input("Ingrese la cantidad de municipios a analizar (número entero): "))
        if n <= 0:
            raise ValueError("El número debe ser mayor que 0")
    except ValueError as e:
        print(f"Error: {e}. Se usará n=5 por defecto.")
        n = 5
    
    # Cálculos generales para Yucatán
    poblacion_total_yuc = yucatan_df["pobtot"].sum()
    poblacion_masculina_yuc = yucatan_df["pobmas"].sum()
    poblacion_femenina_yuc = yucatan_df["pobfem"].sum()
    poblacion_ocupada_yuc = yucatan_df["pocupada_m"].sum() + yucatan_df["pocupada_f"].sum()
    poblacion_desocupada_yuc = yucatan_df["pdesocup_m"].sum() + yucatan_df["pdesocup_f"].sum()
    hogares_jefatura_masculina_yuc = yucatan_df["hogjef_m"].sum()
    hogares_jefatura_femenina_yuc = yucatan_df["hogjef_f"].sum()
    
    print(f"Población total en Yucatán: {poblacion_total_yuc}")
    print(f"Población masculina en Yucatán: {poblacion_masculina_yuc}")
    print(f"Población femenina en Yucatán: {poblacion_femenina_yuc}")
    print(f"Población ocupada en Yucatán: {poblacion_ocupada_yuc}")
    print(f"Población desocupada en Yucatán: {poblacion_desocupada_yuc}")
    print(f"Hogares con jefatura masculina en Yucatán: {hogares_jefatura_masculina_yuc}")
    print(f"Hogares con jefatura femenina en Yucatán: {hogares_jefatura_femenina_yuc}")
    
    # Análisis de municipios y localidades
    numero_municipios = yucatan_df["mun"].nunique()
    numero_localidades = yucatan_df["loc"].nunique()
    
    # Sortear una vez para eficiencia
    df_sorted_asc = yucatan_df.sort_values("pobtot", ascending=True)
    df_sorted_desc = yucatan_df.sort_values("pobtot", ascending=False)
    
    poblaciones_altas = df_sorted_desc["pobtot"].head(n).values
    poblaciones_bajas = df_sorted_asc["pobtot"].head(n).values
    
    print(f"Las {n} localidades con mayor población en Yucatán son:")
    for i in range(n):
        poblacion_alta = poblaciones_altas[i]
        row_alta = df_sorted_desc.iloc[i]
        nombre_alta = row_alta["nom_ent"]
        municipio_alto = row_alta["mun"]
        nombre_mun = row_alta["nom_mun"]
        localidad_alta = row_alta["loc"]
        print(f"{i+1}. Entidad: {nombre_alta}, Municipio: {nombre_mun} ({municipio_alto}), Localidad: {localidad_alta}, Población: {poblacion_alta}\n")
    
    print(f"\nLas {n} localidades con menor población en Yucatán son:\n")
    for i in range(n):
        poblacion_baja = poblaciones_bajas[i]
        row_baja = df_sorted_asc.iloc[i]
        nombre_baja = row_baja["nom_ent"]
        municipio_bajo = row_baja["mun"]
        nombre_mun = row_baja["nom_mun"]
        localidad_baja = row_baja["loc"]
        print(f"{i+1}. Entidad: {nombre_baja}, Municipio: {nombre_mun} ({municipio_bajo}), Localidad: {localidad_baja}, Población: {poblacion_baja}")
    
    # Gráficos
    plt.figure(figsize=(10, 6))
    plt.bar(['Número de Municipios', 'Número de Localidades'], [numero_municipios, numero_localidades], color=['#D3D3D3', "#1fa8b4"])
    plt.title('Número de Municipios y Localidades en Yucatán (2010)')
    plt.xlabel('Categoría')
    plt.ylabel('Cantidad')
    plt.show()
    
    # Gráfico mejorado para n valores
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.bar(range(1, n+1), poblaciones_altas, color='#CFCBCA')
    plt.title(f'{n} Localidades con Mayor Población')
    plt.xlabel('Ranking')
    plt.ylabel('Población')
    
    plt.subplot(1, 2, 2)
    plt.bar(range(1, n+1), poblaciones_bajas, color='#1fa8b4')
    plt.title(f'{n} Localidades con Menor Población')
    plt.xlabel('Ranking')
    plt.ylabel('Población')
    plt.tight_layout()
    plt.show()
    
    print(f"Número de municipios en Yucatán: {numero_municipios}")
    print(f"Número de localidades en Yucatán: {numero_localidades}")
    
    # Return con valores generales y del max/min (primero de las listas)
    return {
        "numero_municipios": numero_municipios,
        "numero_localidades": numero_localidades,
        "poblacion_alta": poblaciones_altas[0],
        "localidad_alta": df_sorted_desc.iloc[0]["loc"],
        "municipio_alto": df_sorted_desc.iloc[0]["mun"],
        "poblacion_baja": poblaciones_bajas[0],
        "localidad_baja": df_sorted_asc.iloc[0]["loc"],
        "municipio_bajo": df_sorted_asc.iloc[0]["mun"],
        "poblacion_total_yuc": poblacion_total_yuc,
        "poblacion_masculina_yuc": poblacion_masculina_yuc,
        "poblacion_femenina_yuc": poblacion_femenina_yuc,
        "poblacion_ocupada_yuc": poblacion_ocupada_yuc,
        "poblacion_desocupada_yuc": poblacion_desocupada_yuc,
        "hogares_jefatura_masculina_yuc": hogares_jefatura_masculina_yuc,
        "hogares_jefatura_femenina_yuc": hogares_jefatura_femenina_yuc
    }

def filtrar_por_municipio(yucatan_df):
    yucatan_df_Merida = yucatan_df[yucatan_df["mun"] == 50]
    yucatan_df_Ticul = yucatan_df[yucatan_df["mun"] == 88]
    yucatan_df_Progreso = yucatan_df[yucatan_df["mun"] == 59]
    return yucatan_df_Merida, yucatan_df_Ticul, yucatan_df_Progreso

def analisis_por_municipio(yucatan_df_Merida, yucatan_df_Ticul, yucatan_df_Progreso):
    # Análisis demográfico de Mérida 2010
    print("Análisis Demográfico de Mérida 2010")
    poblacion_total_merida = yucatan_df_Merida["pobtot"].sum()
    poblacion_masculina_merida = yucatan_df_Merida["pobmas"].sum()
    poblacion_femenina_merida = yucatan_df_Merida["pobfem"].sum()
    poblacion_ocupada_merida = yucatan_df_Merida["pocupada_m"].sum() + yucatan_df_Merida["pocupada_f"].sum()
    poblacion_desocupada_merida = yucatan_df_Merida["pdesocup_m"].sum() + yucatan_df_Merida["pdesocup_f"].sum()
    hogares_jefatura_masculina_merida = yucatan_df_Merida["hogjef_m"].sum()
    hogares_jefatura_femenina_merida = yucatan_df_Merida["hogjef_f"].sum()

    print(f"Población total en Mérida: {poblacion_total_merida}")
    print(f"Población masculina en Mérida: {poblacion_masculina_merida}")
    print(f"Población femenina en Mérida: {poblacion_femenina_merida}")
    print(f"Población ocupada en Mérida: {poblacion_ocupada_merida}")
    print(f"Población desocupada en Mérida: {poblacion_desocupada_merida}")
    print(f"Hogares con jefatura masculina en Mérida: {hogares_jefatura_masculina_merida}")
    print(f"Hogares con jefatura femenina en Mérida: {hogares_jefatura_femenina_merida}")

    # Análisis demográfico de Ticul 2010
    print("\nAnálisis Demográfico de Ticul 2010\n")
    poblacion_total_ticul = yucatan_df_Ticul["pobtot"].sum()
    poblacion_masculina_ticul = yucatan_df_Ticul["pobmas"].sum()
    poblacion_femenina_ticul = yucatan_df_Ticul["pobfem"].sum()
    poblacion_ocupada_ticul = yucatan_df_Ticul["pocupada_m"].sum() + yucatan_df_Ticul["pocupada_f"].sum()
    poblacion_desocupada_ticul = yucatan_df_Ticul["pdesocup_m"].sum() + yucatan_df_Ticul["pdesocup_f"].sum()
    hogares_jefatura_masculina_ticul = yucatan_df_Ticul["hogjef_m"].sum()
    hogares_jefatura_femenina_ticul = yucatan_df_Ticul["hogjef_f"].sum()

    print(f"\nPoblación total en Ticul: {poblacion_total_ticul}\n")
    print(f"Población masculina en Ticul: {poblacion_masculina_ticul}")
    print(f"Población femenina en Ticul: {poblacion_femenina_ticul}")
    print(f"Población ocupada en Ticul: {poblacion_ocupada_ticul}")
    print(f"Población desocupada en Ticul: {poblacion_desocupada_ticul}")
    print(f"Hogares con jefatura masculina en Ticul: {hogares_jefatura_masculina_ticul}")
    print(f"Hogares con jefatura femenina en Ticul: {hogares_jefatura_femenina_ticul}")

    # Análisis demográfico de Progreso 2010
    print("\nAnálisis Demográfico de Progreso 2010\n")
    poblacion_total_progreso = yucatan_df_Progreso["pobtot"].sum()
    poblacion_masculina_progreso = yucatan_df_Progreso["pobmas"].sum()
    poblacion_femenina_progreso = yucatan_df_Progreso["pobfem"].sum()
    poblacion_ocupada_progreso = yucatan_df_Progreso["pocupada_m"].sum() + yucatan_df_Progreso["pocupada_f"].sum()
    poblacion_desocupada_progreso = yucatan_df_Progreso["pdesocup_m"].sum() + yucatan_df_Progreso["pdesocup_f"].sum()
    hogares_jefatura_masculina_progreso = yucatan_df_Progreso["hogjef_m"].sum()
    hogares_jefatura_femenina_progreso = yucatan_df_Progreso["hogjef_f"].sum()

    print(f"Población total en Progreso: {poblacion_total_progreso}")
    print(f"Población masculina en Progreso: {poblacion_masculina_progreso}")
    print(f"Población femenina en Progreso: {poblacion_femenina_progreso}")
    print(f"Población ocupada en Progreso: {poblacion_ocupada_progreso}")
    print(f"Población desocupada en Progreso: {poblacion_desocupada_progreso}")
    print(f"Hogares con jefatura masculina en Progreso: {hogares_jefatura_masculina_progreso}")
    print(f"Hogares con jefatura femenina en Progreso: {hogares_jefatura_femenina_progreso}")

    return {
        "merida": {
            "poblacion_total": poblacion_total_merida,
            "poblacion_masculina": poblacion_masculina_merida,
            "poblacion_femenina": poblacion_femenina_merida,
            "poblacion_ocupada": poblacion_ocupada_merida,
            "poblacion_desocupada": poblacion_desocupada_merida,
            "hogares_jefatura_masculina": hogares_jefatura_masculina_merida,
            "hogares_jefatura_femenina": hogares_jefatura_femenina_merida
        },
        "ticul": {
            "poblacion_total": poblacion_total_ticul,
            "poblacion_masculina": poblacion_masculina_ticul,
            "poblacion_femenina": poblacion_femenina_ticul,
            "poblacion_ocupada": poblacion_ocupada_ticul,
            "poblacion_desocupada": poblacion_desocupada_ticul,
            "hogares_jefatura_masculina": hogares_jefatura_masculina_ticul,
            "hogares_jefatura_femenina": hogares_jefatura_femenina_ticul
        },
        "progreso": {
            "poblacion_total": poblacion_total_progreso,
            "poblacion_masculina": poblacion_masculina_progreso,
            "poblacion_femenina": poblacion_femenina_progreso,
            "poblacion_ocupada": poblacion_ocupada_progreso,
            "poblacion_desocupada": poblacion_desocupada_progreso,
            "hogares_jefatura_masculina": hogares_jefatura_masculina_progreso,
            "hogares_jefatura_femenina": hogares_jefatura_femenina_progreso
        }
    }


def comparar_diferencias(general_results, municipios_results):
    print("\nComparación de Diferencias entre Municipios y Yucatán General")
    
    opciones = {
        'M': 'merida',
        'T': 'ticul',
        'P': 'progreso',
    }

    seleccion = input("Indique las comparaciones que desea ver (M, T, P) Merida, Ticul, Progreso:  ").upper()
    

    for opcion in seleccion:
        if opcion in opciones:
            municipio_key = opciones[opcion]
            municipio_nombre = municipio_key.capitalize()
            
            data = {
                "Métrica": [
                    "Población Total",
                    "Población Masculina",
                    "Población Femenina",
                    "Población Ocupada",
                    "Población Desocupada",
                    "Hogares Jefatura Masculina",
                    "Hogares Jefatura Femenina"
                ],
                "Yucatán": [
                    general_results["poblacion_total_yuc"],
                    general_results["poblacion_masculina_yuc"],
                    general_results["poblacion_femenina_yuc"],
                    general_results["poblacion_ocupada_yuc"],
                    general_results["poblacion_desocupada_yuc"],
                    general_results["hogares_jefatura_masculina_yuc"],
                    general_results["hogares_jefatura_femenina_yuc"]
                ],
                municipio_nombre: [
                    municipios_results[municipio_key]["poblacion_total"],
                    municipios_results[municipio_key]["poblacion_masculina"],
                    municipios_results[municipio_key]["poblacion_femenina"],
                    municipios_results[municipio_key]["poblacion_ocupada"],
                    municipios_results[municipio_key]["poblacion_desocupada"],
                    municipios_results[municipio_key]["hogares_jefatura_masculina"],
                    municipios_results[municipio_key]["hogares_jefatura_femenina"]
                ],
                f"{municipio_nombre} (% de Yuc)": [
                    f"{(municipios_results[municipio_key]['poblacion_total'] / general_results['poblacion_total_yuc'] * 100):.2f}%",
                    f"{(municipios_results[municipio_key]['poblacion_masculina'] / general_results['poblacion_masculina_yuc'] * 100):.2f}%",
                    f"{(municipios_results[municipio_key]['poblacion_femenina'] / general_results['poblacion_femenina_yuc'] * 100):.2f}%",
                    f"{(municipios_results[municipio_key]['poblacion_ocupada'] / general_results['poblacion_ocupada_yuc'] * 100):.2f}%",
                    f"{(municipios_results[municipio_key]['poblacion_desocupada'] / general_results['poblacion_desocupada_yuc'] * 100):.2f}%",
                    f"{(municipios_results[municipio_key]['hogares_jefatura_masculina'] / general_results['hogares_jefatura_masculina_yuc'] * 100):.2f}%",
                    f"{(municipios_results[municipio_key]['hogares_jefatura_femenina'] / general_results['hogares_jefatura_femenina_yuc'] * 100):.2f}%"
                ]
            }
            
            df = pd.DataFrame(data)
            print(f"\n--- Comparación Yucatán vs {municipio_nombre} ---\n")
            print(df.to_string(index=False))
            
            plt.figure(figsize=(10, 6))
            plt.bar(['Yucatán', municipio_nombre], [
                general_results["poblacion_total_yuc"],
                municipios_results[municipio_key]["poblacion_total"]
            ], color=['#D3D3D3', "#1fa8b4"])
            plt.title(f'Población Total: Yucatán vs {municipio_nombre} (2010)')
            plt.xlabel('Categoría')
            plt.ylabel('Cantidad')
            plt.show()
        
        else:
            print(f"Opción inválida: {opcion}. Solo se permiten M, T o P.")


# Ejemplo de ejecución integrada (descomentar para usar)
# general_results = analisis_general(yucatan_df)
# merida_df, tizimin_df, progreso_df = filtrar_por_municipio(yucatan_df)
# municipios_results = analisis_por_municipio(merida_df, tizimin_df, progreso_df)
# comparar_diferencias(general_results, municipios_results)

def main():
    print("\n--- Análisis General ---\n")
    general_results = analisis_general(yucatan_df)
    print("\n--- Filtrado de Municipios (Mérida, Ticul, Progreso) ---\n")
    merida_df, ticul_df, progreso_df = filtrar_por_municipio(yucatan_df)
    municipios_results = analisis_por_municipio(merida_df, ticul_df, progreso_df)
    print("\n--- Comparación de Diferencias ---\n")
    comparar_diferencias(general_results, municipios_results)

if __name__ == "__main__":
    main()

# Análisis de los resultados
# En general, se observa que Mérida tiene un mayor acceso a la tecnología en comparación con San Diego Cutz.
# Esto es evidente en las gráficas de barras, donde Mérida muestra porcentajes más altos en todas las categorías de tecnología.
# Además, los gráficos de pastel destacan la diferencia en el acceso a la tecnología entre las dos localidades, con Mérida mostrando una mayor proporción de hogares con acceso a tecnología.

ModuleNotFoundError: No module named 'pandas'

In the 2010 census for Yucatán, the total population was approximately 1.96 million, with a slight female majority (50.7% women vs. 49.3% men), a trend that intensified in Mérida (51.2% women in 2020) due to urban migration, while Ticul and Progreso show balance or male majority (50.5% and 51.2% men, respectively) owing to rural and coastal economies. In occupation, men account for 60-70% of the employed (78.5% male participation rate in 2020), with women facing 1.5-2 times higher unemployment (4.1% vs. 2.5%) and elevated sub-occupation due to caregiving roles; household headship is predominantly male (75-85%), with only 22-30% female, reflecting patriarchal structures that limit female empowerment in municipalities like Ticul (82% male).
This dynamic is mirrored across the Yucatán Peninsula (Yucatán, Quintana Roo, Campeche), with 5.1 million inhabitants in 2020 and women at 51%, yet female labor participation at only 45-46% (gap of 28.8 percentage points in 2025) and headship at 24%, exacerbated by tourism in Quintana Roo (women 55% in services but 60% informal).

The hypothesis is that, based on 2010 census data for Yucatán, the persistent gender gaps in occupation and household headship—where men dominate employment (60-70%) and headship (75-85%), while women face higher unemployment (1.5-2 times greater) and limited empowerment—perpetuate regional inequalities, potentially reducing economic productivity by 10-15% due to underutilized female potential. This effect likely varies by location, with urban areas like Mérida showing greater female participation (though still constrained) and rural zones like Ticul exhibiting stronger patriarchal structures, suggesting that policy interventions, such as localized childcare support, must be tailored to specific municipal contexts to effectively address these disparities.
